In [1]:
import pandas as pd
import numpy as np 
import zipcodes

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pathlib
import sys
sys.path.append(str(pathlib.Path().absolute().parent))

In [2]:
clean_zip = pd.read_pickle('../data/interim/ca-zip-w-city-ts.pickle')

In [57]:
a=clean_zip[clean_zip['ZipCode_int'].notna()]['ZipCode_int'].unique()
a[1:100]
clean_zip.columns

Index(['Date', 'ZipCode_int', 'ZipCode_str', 'City', 'County', 'Lat', 'Long',
       'MedianListingPrice_1Bedroom', 'MedianListingPrice_2Bedroom',
       'MedianListingPrice_3Bedroom', 'MedianListingPrice_4Bedroom',
       'MedianListingPrice_5BedroomOrMore', 'MedianListingPrice_AllHomes',
       'MedianListingPrice_CondoCoop', 'MedianListingPrice_DuplexTriplex',
       'MedianListingPrice_SingleFamilyResidence',
       'MedianRentalPrice_1Bedroom', 'MedianRentalPrice_2Bedroom',
       'MedianRentalPrice_3Bedroom', 'MedianRentalPrice_4Bedroom',
       'MedianRentalPrice_5BedroomOrMore', 'MedianRentalPrice_AllHomes',
       'MedianRentalPrice_CondoCoop', 'MedianRentalPrice_DuplexTriplex',
       'MedianRentalPrice_MultiFamilyResidence5PlusUnits',
       'MedianRentalPrice_SingleFamilyResidence', 'MedianRentalPrice_Studio',
       'RegionName', 'MetroName', 'StateName', 'CensusRegion',
       'Unique_City_ID', 'State'],
      dtype='object')

Take 1 - Zip

In [21]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '96161']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianListingPrice_2Bedroom': 'y'})

2020-03-26
2017-12-31
27


,ds,ZipCode_int,ZipCode_str,City,County,Lat,Long,MedianListingPrice_1Bedroom,y,MedianListingPrice_3Bedroom,...,MedianRentalPrice_DuplexTriplex,MedianRentalPrice_MultiFamilyResidence5PlusUnits,MedianRentalPrice_SingleFamilyResidence,MedianRentalPrice_Studio,RegionName,MetroName,StateName,CensusRegion,Unique_City_ID,State
275265,2014-04-30,96161,96161,Truckee,Nevada,39.3340,-120.1942,NaN,299000.0,449000.0,...,NaN,NaN,NaN,NaN,6057.0,"Truckee, CA",California,West,truckeenevadaca,CA
276568,2014-05-31,96161,96161,Truckee,Nevada,39.3340,-120.1942,NaN,299000.0,475000.0,...,NaN,NaN,NaN,NaN,6057.0,"Truckee, CA",California,West,truckeenevadaca,CA
277871,2014-06-30,96161,96161,Truckee,Nevada,39.3340,-120.1942,NaN,323000.0,495000.0,...,NaN,NaN,NaN,NaN,6057.0,"Truckee, CA",California,West,truckeenevadaca,CA
279174,2014-07-31,96161,96161,Truckee,Nevada,39.3340,-120.1942,NaN,318000.0,487000.0,...,NaN,NaN,NaN,NaN,6057.0,"Truckee, CA",California,West,truckeenevadaca,CA
280477,2014-08-31,96161,96161,Truckee,Nevada,39.3340,-120.1942,NaN,325000.0,479000.0,...,NaN,NaN,NaN,NaN,6057.0,"Truckee, CA",California,West,truckeenevadaca,CA


In [25]:
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()

In [27]:
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


5.927980279481769

Run 2

In [30]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '96146']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianListingPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.


2020-03-26
2017-12-31
27


14.730727670342821

Run 3

In [36]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '90003']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianListingPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 11.


2020-03-26
2017-12-31
27


3.091185810097186

In [53]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '90044']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianListingPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


2020-03-26
2017-12-31
27


18.679992996896715

In [64]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '90292']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianListingPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2020-03-26
2017-12-31
27


13.465537182138501

In [65]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
20,2017-08-31,1.135599e+06,1.058668e+06,1.214880e+06
21,2017-09-30,1.138984e+06,1.057984e+06,1.219752e+06
22,2017-10-31,1.137598e+06,1.057921e+06,1.218995e+06
23,2017-11-30,1.149578e+06,1.065849e+06,1.231547e+06
24,2017-12-31,1.177936e+06,1.100881e+06,1.263117e+06


RENTAL

In [58]:
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '96161']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianRentalPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

2020-03-26
2017-12-31
27


ValueError: Dataframe has less than 2 non-NaN rows.

In [62]:
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in clean_zip['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)
zip_ts = clean_zip.loc[clean_zip['ZipCode_str'] == '90292']
my_data_1 = zip_ts.rename(columns={'Date': 'ds',
                        'MedianRentalPrice_2Bedroom': 'y'})
split_date = '2015-11-30'
zip_train = my_data_1.loc[my_data_1['ds'] <= split_date].copy()
zip_test = my_data_1.loc[my_data_1['ds'] > split_date].copy()
my_model = Prophet(interval_width=0.95)
my_model.fit(zip_train)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(zip_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

mean_absolute_percentage_error(y_true=zip_test['y'],
                   y_pred=forecast['yhat'])

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2020-03-26
2017-12-31
27


5.187636678813009

In [63]:
Zip_codes = [90292, 96146, 96161, 90044, 90003]
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
20,2017-08-31,4104.922748,3273.928867,4828.955825
21,2017-09-30,4086.431701,3200.634771,4916.611846
22,2017-10-31,3997.610580,3032.238101,4860.628434
23,2017-11-30,3887.733636,2890.424814,4816.097491
24,2017-12-31,3868.330389,2787.465413,4874.307983
